# Packages

In [118]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from fanalysis.mca import MCA
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
import scipy

# Utilisation du module functions

In [119]:
from mlops_functions import functions
from mlops_functions import stats

# Lecture de données

In [120]:
from mlops_functions.functions import load_data

# Louise
#base = functions.load_data("//ad.univ-lille.fr/Etudiants/Homedir3/147794/Documents/M2/Rendu_MLOPS/athlete_events.csv")


# Cyrielle
base = functions.load_data("/Users/Cyrie/OneDrive/Bureau/M2_DS/S1/MLOPS/Rendu_MLOPS/athlete_events.csv")


# Filtre sur Summer

In [121]:
from mlops_functions.functions import filter_summer_season
# Filtre sur la saison d'été
base_summer = functions.filter_summer_season(base)


# Valeurs manquantes et abbérantes du df base_summer

In [ ]:
#valeurs manquantes pour le df base summer
valeurs_manquantes_par_variable1 = base_summer.isnull().sum()

# Affichage du résultat pour le df base summer
print(valeurs_manquantes_par_variable1)

In [ ]:
#valeur abbérentes
# Création de la figure et des axes
plt.figure(figsize=(8, 6))

###################AGE#####################
# Création de la boîte à moustaches 
sns.boxplot(x=base_summer['Medal'], y=base_summer['Age'], color='#008030')

# Titres et labels
plt.title('Boîte à moustaches de la variable age selon la médaille')
plt.xlabel('Médaille')
plt.show()

###################Height####################
# Création de la boîte à moustaches 
sns.boxplot(x=base_summer['Medal'], y=base_summer['Height'], color='#008080')

# Titres et labels
plt.title('Boîte à moustaches de la variable taille selon la médaille')
plt.xlabel('Médaille')
plt.show()


###################Weight####################
# Création de la boîte à moustaches 
sns.boxplot(x=base_summer['Medal'], y=base_summer['Weight'], color='#008030')

# Titres et labels
plt.title('Boîte à moustaches de la variable poids selon la médaille')
plt.xlabel('Médaille')
plt.show()




In [ ]:
#lecture des valeurs abbérantes
# Filtre sur les enregistrements où l'âge est supérieur à 40 et inférieur à 11 ans
base_summer_age_11_40 = base_summer[(base_summer['Age'] > 40) | (base_summer['Age'] < 11)]
print("DataFrame pour l'âge entre 11 et 40 ans inclus:")
print(base_summer_age_11_40)

# Filtre sur les enregistrements où la taille est inférieure à 145 cm et supérieure à 210 cm
base_summer_height_145_210 = base_summer[(base_summer['Height'] > 210) | (base_summer['Height'] < 145)]
print("\nDataFrame pour la taille entre 145 et 210 cm inclus:")
print(base_summer_height_145_210)

# Filtre sur les enregistrements où le poids est inférieur à 35 kg et supérieur à 118 kg
base_summer_weight_35_118 = base_summer[(base_summer['Weight'] > 118) | (base_summer['Weight'] < 35)]
print("\nDataFrame pour le poids entre 35 et 118 kg inclus:")
print(base_summer_weight_35_118)


# Regroupements

In [ ]:
#voir la répartiton des variables quantitatives avant le regroupement
###################YEAR####################
plt.figure(figsize=(8, 6))
sns.histplot(base_summer['Year'], bins=30, kde=True, color='#008030', stat='density', element='bars', common_norm=False)  # Utilisation du code hexadécimal pour un bleu canard foncé
sns.kdeplot(base_summer['Year'], color='red', linewidth=2)  # Courbe KDE en rouge
plt.title('Répartition de la variable année ')
plt.xlabel('Année')
plt.ylabel('Densité')
plt.show()

###################AGE#####################
plt.figure(figsize=(8, 6))
sns.histplot(base_summer['Age'], bins=30, kde=True, color='#008080', stat='density', element='bars', common_norm=False)  # Utilisation du code hexadécimal pour un bleu canard foncé
sns.kdeplot(base_summer['Age'], color='red', linewidth=2)  # Courbe KDE en rouge
plt.title('Répartition de la variable age ')
plt.xlabel('Age')
plt.ylabel('Densité')
plt.show()

###################HEIGHT#####################
plt.figure(figsize=(8, 6))
sns.histplot(base_summer['Height'], bins=30, kde=True, color='#008030', stat='density', element='bars', common_norm=False)  # Utilisation du code hexadécimal pour un bleu canard foncé
sns.kdeplot(base_summer['Height'], color='red', linewidth=2)  # Courbe KDE en rouge
plt.title('Répartition de la variable taille ')
plt.xlabel('Taille')
plt.ylabel('Densité')
plt.show()

###################WEIGHT#####################
plt.figure(figsize=(8, 6))
sns.histplot(base_summer['Weight'], bins=30, kde=True, color='#008030', stat='density', element='bars', common_norm=False)  # Utilisation du code hexadécimal pour un bleu canard foncé
sns.kdeplot(base_summer['Weight'], color='red', linewidth=2)  # Courbe KDE en rouge
plt.title('Répartition de la variable poids ')
plt.xlabel('Poids')
plt.ylabel('Densité')
plt.show()

In [ ]:
from mlops_functions.functions import create_age_classes, create_height_classes, create_weight_classes

# Création des classes d'âge
age_classes = create_age_classes(base_summer)
print(age_classes.head(5))

# Création des classes de taille
height_classes = create_height_classes(base_summer)
print(height_classes.head(5))

# Création des classes de poids
weight_classes = create_weight_classes(base_summer)
print(weight_classes.head(70))

In [ ]:
#afficher les frequences de modalites pour les variables regroupes
###################AGE#####################
print("Fréquences de modalités pour la Classe âge:")
print(base_summer['Classe_age'].value_counts())

###################HEIGHT#####################
print("\nFréquences de modalités pour la Classe taille:")
print(base_summer['Classe_height'].value_counts())

###################WEIGHT#####################
print("\nFréquences de modalités pour la Classe poids:")
print(base_summer['Classe_weight'].value_counts())


#fonction pour afficher les fréquences en pourcentage
def print_percentage_counts(column_name):
    percentages = base_summer[column_name].value_counts(normalize=True) * 100
    print(f"Fréquences en pourcentage pour la colonne '{column_name}':")
    print(percentages)

# Afficher les fréquences en pourcentage pour la classe d'âge
print_percentage_counts('Classe_age')

# Afficher les fréquences en pourcentage pour la classe de taille
print("\n")
print_percentage_counts('Classe_height')

# Afficher les fréquences en pourcentage pour la classe de poids
print("\n")
print_percentage_counts('Classe_weight')


# Filtre les années 2000 et suppression de variable

In [ ]:
from mlops_functions.functions import filter_by_year

# Filtrer sur l'année supérieure ou égale à 2000
base_summer_2000 = filter_by_year(base_summer)
print(base_summer_2000.head())

In [ ]:
from mlops_functions.functions import drop_columns

# Filtrer sur l'année supérieure ou égale à 2000
base_summer_2000 = filter_by_year(base_summer)
print("Avant suppression des colonnes :")
print(base_summer_2000.head())

# Supprimer les variables 'Team' et 'Season'
columns_to_drop = ['Team', 'Season', 'Age', 'Height', 'Weight']
base_summer_2000 = drop_columns(base_summer_2000, columns_to_drop)
print("\nAprès suppression des colonnes :")
print(base_summer_2000.head())


# Valeurs manquantes du df base_summer_2000

In [ ]:
from mlops_functions.functions import display_missing_values

# Filtrer sur l'année supérieure ou égale à 2000
base_summer_2000 = filter_by_year(base_summer)
print("Avant suppression des colonnes :")
print(base_summer_2000.head())

# Calculer et afficher les valeurs manquantes
valeurs_manquantes_par_variable2 = display_missing_values(base_summer_2000)


In [ ]:
from mlops_functions.functions import drop_missing_values

# Filtrer sur l'année supérieure ou égale à 2000
base_summer_2000 = filter_by_year(base_summer)
print("Avant suppression des colonnes :")
print(base_summer_2000.head())

# Supprimer les valeurs manquantes pour les colonnes spécifiques
columns_to_check = ['Classe_age', 'Classe_height', 'Classe_weight']
base_summer_2000_net = drop_missing_values(base_summer_2000, columns_to_check)
print("\nAprès suppression des valeurs manquantes :")
print(base_summer_2000_net.head())

In [ ]:
#nombre de ligne de notre df base après filtre du df base summer
nombre_de_lignes3 = base_summer_2000_net.shape[0]

#affiche le nombre de lignes du df base summer 2000
print("Nombre de lignes :", nombre_de_lignes3)

# Statistiques descriptives

In [127]:
from mlops_functions.stats import plot_sex_pie_chart, plot_age_class_pie_chart, plot_count_percentage_bar_chart, plot_noc_distribution

# Exemple d'utilisation
plot_sex_pie_chart(base_summer_2000_net, 'Répartition des sexes')
plot_age_class_pie_chart(base_summer_2000_net, 'Répartition des âges')
plot_height_class_pie_chart(base_summer_2000_net, 'Classe_height', 'Répartition des tailles')
plot_count_percentage_bar_chart(base_summer_2000_net, 'Classe_height', 'Répartition des tailles')
plot_pie_chart(base_summer_2000_net, 'Classe_weight', 'Répartition des poids')
plot_count_percentage_bar_chart(base_summer_2000_net, 'Classe_weight', 'Répartition des poids')
plot_noc_distribution(base_summer_2000_net)


ImportError: cannot import name 'plot_sex_pie_chart' from 'mlops_functions.stats' (c:\Users\Cyrie\OneDrive\Bureau\M2_DS\S1\MLOPS\Rendu_MLOPS\mlops_functions\stats.py)

## statistique univarié

In [ ]:
colors = ['#008030', '#008080', '#00A86B', '#4CAF50', '#7CFC00']

# Statistique sur la variable sex 

plt.figure(figsize=(6,6))
base_summer_2000_net['Sex'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',shadow=True,startangle=15, colors = colors)
plt.title('Répartition des sexes')
plt.show()

In [ ]:
# Statistique sur la variable classe_age

plt.figure(figsize=(6,6))
base_summer_2000_net['Classe_age'].value_counts().plot.pie(colors = colors,autopct='%1.1f%%')
plt.title('Répartition des âges')
plt.show()

In [ ]:
# Statistique sur la variable classe_height

plt.figure(figsize=(6,6))
base_summer_2000_net['Classe_height'].value_counts().plot.pie(colors = colors, autopct='%1.1f%%')
plt.title('Répartition des tailles')
plt.show()

In [ ]:
# Statistique sur la variable height

plt.figure(figsize=(12, 6))

# Comptez les occurrences de chaque catégorie
total = float(len(base_summer_2000_net["Classe_height"]))

# Utilisez sns.countplot pour afficher le graphique à barres
ax = sns.countplot(x="Classe_height", data=base_summer_2000_net, palette='viridis')

# Modifiez l'axe y pour être en pourcentage
def percent_formatter(x, pos):
    return f'{(x / total):.0%}'

ax.yaxis.set_major_formatter(FuncFormatter(percent_formatter))

# Ajoutez les pourcentages au centre des barres
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2., height / 2,
            '{:.0%}'.format(height / total), ha="center")

plt.title('Répartition des tailles')
plt.xlabel("Classe_height")
plt.ylabel('Pourcentage')
plt.xticks(rotation=45)
plt.show()



In [ ]:
# Statistique sur la variable weight

plt.figure(figsize=(6,6))
base_summer_2000_net['Classe_weight'].value_counts().plot.pie(colors = colors, autopct='%1.1f%%')
plt.title('Répartition des poids')
plt.show()

In [ ]:
# Statistique sur la variable weight

plt.figure(figsize=(12, 6))

# Comptez les occurrences de chaque catégorie
total = float(len(base_summer_2000_net["Classe_weight"]))

# Utilisez sns.countplot pour afficher le graphique à barres
ax = sns.countplot(x="Classe_weight", data=base_summer_2000_net, palette='viridis')

# Modifiez l'axe y pour être en pourcentage
def percent_formatter(x, pos):
    return f'{(x / total):.0%}'

ax.yaxis.set_major_formatter(FuncFormatter(percent_formatter))

# Ajoutez les pourcentages au centre des barres
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2., height / 2,
            '{:.0%}'.format(height / total), ha="center")

plt.title('Répartition des poids')
plt.xlabel("Classe_xeight")
plt.ylabel('Pourcentage')
plt.xticks(rotation=45)
plt.show()



In [ ]:
# Statistique sur la variable NOC 

# Créer un DataFrame pour le nombre d'individus par pays
tableau_noc = base_summer_2000_net.groupby('NOC').size().reset_index(name='Nombre d\'individus')

# Trier le tableau par le nombre d'individus de manière décroissante
tableau_noc_trie = tableau_noc.sort_values(by='Nombre d\'individus', ascending=False)

# Sélectionner les 16 premiers
les_16_premiers = tableau_noc_trie.head(16)

# Afficher les 16 premiers pays les plus représentés
les_16_premiers


## statistique bivarié

In [ ]:
def create_bar_plot(base_summer_2000_net, x_variable, target_variable='Medal'):

    # Create a bar plot
    plt.figure(figsize=(12, 6))
    sns.countplot(x=x_variable, hue=target_variable, data=base_summer_2000_net, palette='viridis')
    plt.title(f'Répartition des {target_variable} selon leurs {x_variable}')
    plt.xlabel(x_variable)
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.legend(title=target_variable)
    plt.show()

# Example usage:
# Assuming your DataFrame is named df
create_bar_plot(base_summer_2000_net, 'Sex')
create_bar_plot(base_summer_2000_net, 'Classe_age')
create_bar_plot(base_summer_2000_net, 'Classe_height')
create_bar_plot(base_summer_2000_net, 'Classe_weight')
create_bar_plot(base_summer_2000_net, 'NOC')
create_bar_plot(base_summer_2000_net, 'Games')
create_bar_plot(base_summer_2000_net, 'Sport')



# Corrélation entre les variables - V de Cramer

In [9]:
# Sélectionnez vos variables catégorielles
categorical_vars = ['Name', 'Sex', 'NOC', 'Games', 'City', 'Sport',
                    'Event', 'Medal', 'Classe_age', 'Classe_height', 'Classe_weight']

In [ ]:
# Cette fonction permet de calculer le tableau de contingenance. Au vu de nos données le résultat est long a sortir, donc nou conseillons de ne pas le lancer. 

from mlops_functions.functions import calculate_cramer_v, plot_cramer_matrix

# Calculez le V de Cramer et affichez la matrice de corrélation
# cramer_matrix = calculate_cramer_v(base_summer_2000_net, categorical_vars)
# plot_cramer_matrix(cramer_matrix)


# Faire de nouveaux regroupement et suppression des variables utilisées pour la suite de l'analyse

In [11]:
base_ACM = base_summer_2000_net


In [ ]:
from mlops_functions.functions import transform_base_ACM

base_ACM = transform_base_ACM(base_summer_2000_net)

# Affichage des colonnes après transformation
print(base_ACM.columns)

In [ ]:
#remplacer les valeurs manquantes de la variable cibe par "pas de médaille"
base_ACM['Medal'].fillna('Pas_de_médaille', inplace=True)
base_ACM.isnull().sum()

# ACM 

In [ ]:
from mlops_functions.functions import analyze_acm

base_ACM = analyze_acm(base_ACM)
#remplacer les valeurs manquantes de la variable cibe par "pas de médaille"
#base_ACM['Medal'].fillna('Pas_de_médaille', inplace=True)
#print(base_ACM)


In [ ]:
# pip install fanalysis
# utilisation du package fanalysis 

#acm = MCA()
#acm.fit(base_ACM.values)

In [ ]:
# afficher les valeurs propres 
#acm.eig_

In [24]:
#eigenvalues = acm.eig_

In [ ]:
# Obtenir les valeurs propres associées à chaque composante principale
#eigenvalues = acm.eig_

# Calculer le pourcentage de chaque valeur propre
#total_variance = sum(eigenvalues[0])
#percentage_var = [(value / total_variance) * 100 for value in eigenvalues[0]]

# Utiliser une palette de couleurs Seaborn et inverser l'ordre
#sns.set(style="whitegrid")
#palette = sns.color_palette("BuGn", len(percentage_var))
#palette = palette[::-1]  # Inverser l'ordre des couleurs

# Tracer le diagramme en barres des valeurs propres en pourcentage
#plt.figure(figsize=(10, 6))
#bars = plt.bar(range(1, len(percentage_var) + 1), percentage_var, color=palette, edgecolor='black')
#plt.xlabel('Composante Principale')
#plt.ylabel('Pourcentage de Variance Expliquée')
#plt.title('Diagramme en Barres des Valeurs Propres en Pourcentage - ACM')

# Définir les ticks de l'axe x avec des valeurs entières
#plt.xticks(range(1, len(percentage_var) + 1))

# Ajouter les étiquettes de valeur uniquement pour les deux premières barres
#for i, (bar, value) in enumerate(zip(bars, percentage_var)):
#    if i < 2:
#        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.2, f'{value:.2f}%',
#                 ha='center', va='bottom', color='black', fontsize = 8)

#plt.show()

In [ ]:
# indormation que procure l'ACM 
#info_col = acm.col_topandas()
#info_col.columns

In [ ]:
# Coordonnée des modalité pour l'axe 1 et 2
#coord_col = info_col[['col_coord_dim1', 'col_coord_dim2']]
#print(coord_col)

In [ ]:
# Contribution des modalités pour l'axe 1 et 2
#contrib_col = pd.DataFrame(info_col[['col_contrib_dim1', 'col_contrib_dim2']])
#contrib_col

In [ ]:
#ACM -projection des colonnes
#import matplotlib.pyplot as plt

#fig, ax = plt.subplots(figsize=(7, 7))
#ax.axis([-2.2, +2.2, -1.2, +1.5])
#ax.plot([-2.2, +2.2], [0, 0], color="silver", linestyle="--")
#ax.plot([0, 0], [-2.2, +2.2], color='silver', linestyle="--")
#ax.set_xlabel('Dim.1')
#ax.set_ylabel('Dim.2')
#plt.title("Modalité")

#for x, y, lbl in zip(coord_col.iloc[:, 0], coord_col.iloc[:, 1], coord_col.index):
#    ax.text(x, y, lbl, horizontalalignment='center', verticalalignment='center', fontsize=7)

#plt.show()


In [ ]:
#ACM -projection en couleur
#acm.mapping_col(num_x_axis = 1 , num_y_axis = 2)

# Modélisation

In [35]:
from mlops_functions.functions import preprocess_and_split_data

# Prétraiter et diviser les données
base_model = base_ACM
X_train_encoded, X_test_encoded, y_train, y_test = preprocess_and_split_data(base_model)


Valeurs uniques avant la modification : ['Pas_de_médaille' 'medaille']
Valeurs uniques après la modification : ['Pas_de_médaille' 'medaille']


In [58]:
from mlops_functions.functions import define_preprocessor
# Utiliser la fonction define_preprocessor pour obtenir preprocessor et cat_columns
preprocessor, cat_columns = define_preprocessor()

## Régression logistique

In [ ]:
from mlops_functions.functions import train_and_evaluate_logistic_regression
LG = train_and_evaluate_logistic_regression(X_train_encoded, X_test_encoded, y_train, y_test)

## Random Forest 

In [ ]:
from mlops_functions.functions import train_and_evaluate_random_forest
trained_model_RF = train_and_evaluate_random_forest(X_train_encoded, X_test_encoded, y_train, y_test)

# KNN

In [ ]:
from mlops_functions.functions import choose_k_and_evaluate_knn
trained_model_KNN = choose_k_and_evaluate_knn(X_train_encoded, X_test_encoded, y_train, y_test)

# interprétation du modèle Regression logisitique

In [61]:
from mlops_functions.functions import analyze_logistic_regression

interpret = analyze_logistic_regression(LG, preprocessor, X_train_encoded, cat_columns)

# permet d'obtenir les coefficients directeurs associé aux modalités

#coefficients = LG.coef_
#coefficients

AttributeError: 'ColumnTransformer' object has no attribute 'transformers_'

In [39]:
# permet d'obtenir les coefficients directeurs associé aux modalités

coefficients = LG.coef_
coefficients

array([[-0.3997242 , -1.3302836 ,  1.57874587,  0.6628577 ,  0.9293529 ,
         0.97241796,  1.10450648,  0.51699593,  1.02458601,  0.31638146,
         0.54152503,  0.22676924,  1.35247967,  0.98851018,  0.0887227 ,
        -0.2982972 , -0.10728899,  0.13377636, -0.10921965,  0.25590161,
         0.1301756 ,  0.02527832,  0.19080879]])

In [41]:
# Obtenir les noms des colonnes après la transformation
column_names = preprocessor.named_transformers_['cat'].get_feature_names_out(cat_columns)

# Associer les coefficients aux noms des colonnes
coefficients_dict = dict(zip(column_names, LG.coef_[0]))

# Afficher les coefficients avec les noms des colonnes associés
for variable, coef in coefficients_dict.items():
    print(f"{variable}: {coef}")

AttributeError: 'ColumnTransformer' object has no attribute 'transformers_'

In [ ]:
# permet d'obtenir le coefficient de l'individu de référence

intercept = LG.intercept_
intercept

In [ ]:
# Obtenez les probabilités prédites
predicted_probabilities = LG.predict_proba(X_train_encoded)

# Calculez les z-scores des coefficients
z_scores = np.zeros(shape=(len(LG.coef_[0]),))
for index, coef in enumerate(LG.coef_[0]):
    z_scores[index] = coef / np.std(predicted_probabilities[:, 1])

# Convertissez les z-scores en p-values approximatives
p_values = scipy.stats.norm.sf(abs(z_scores))
p_values

# Créez un dictionnaire pour associer les noms de colonnes après la transformation aux p-values
p_values_dict = dict(zip(column_names, p_values))

# Affichez les associations entre les modalités et les p-values
for column_name, p_value in p_values_dict.items():
    if p_value < 0.001:
        print(f"{column_name}: <0.001")
    else:
        print(f"{column_name}: {p_value:.3f}")
